## MLE estimation of a simple macroeconomic model

You can observe time series data in an economy for the following variables $(c_t, k_t, w_t, r_t)$. The first line of this file contains variable labels. These data have 100 periods, which are quarterly (25 years). Suppose you think the data are generated by a pricess similar to the Brock and Mirman (1972). A simplified set of characterizing equations of the Brock and Mirman model are the following:
\begin{align}
c_t^{-1} - \beta E[r_{t+1}(c_{t+1})^{-1}] & = 0 && \text{(1)} \\
c_t - k_{t+1} - w_t - r_t k_t & = 0 && \text{(2)} \\
w_t - (1-\alpha)e^{z_t}k_t^\alpha & = 0 && \text{(3)} \\
r_t - \alpha e^{z_t} k_t^{\alpha - 1} & = 0 && \text{(4)} \\
z_t & = \rho z_{t-1} + (1-\rho)\mu + \epsilon_t && \text{(5)} \\
\epsilon_t & \sim N(0,\sigma^2)
\end{align}
The variable $c_t$ is aggregate consumption in period $t$. $k_{t+1}$ is total household savings and investment in period $t$ for which they receive a return in the next period (this model assumes full depreciation of capital). The wage per unit of labor in period $t$ is $w_t$ adn the interest rate or rate of return on investment is $r_t$. Total factor productivity is $z_t$, which follows an AR(1) process. The rest of the symbols in the equations are parameters that must be estimated:
$$ (\alpha, \beta, \rho, \mu, \sigma) $$
The constraints on these parameters are:
$$ \alpha, \beta \in (0,1), \quad \mu,\sigma > 0, \quad \rho \in (-1,1) $$
Assume that the first observation in the data file is $t=1$. Let $k_1$ be the first observation in the data file for $k_t$. Assume that $z_0 = \mu$, so that $z_1 = \mu$. Assume that the discount factor is known to be $\beta = 0.99$.

In [ ]:
# Load Modules
import numpy as np
import scipy.stats as sts
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as opt
import scipy.special as spc
import scipy.integrate as integrate

In [ ]:
# Import Data
macroseries = pd.read_csv('MacroSeries.csv')

(a) Use the data $(w_t,k_t)$ and equations (3) and (5) to estimate the four parameters $(\alpha, \rho, \mu, \sigma)$ by maximum likelihood. Given a guess for the parameters $(\alpha, \rho, \mu, \sigma)$, you can use the two variables from the data $(w_t,k_t)$ and (3) to back out a series for $z_t$. You can then use equation (5) to compute the probability of each 
$$ z_t \sim N(\rho z_{t-1} + (1-\rho) \mu, \sigma^2) $$
The maximum likelihood estimate $(\hat{\alpha},\hat{\rho},\hat{\mu},\hat{\sigma})$ maximzies the likelihood function of that normal distribution of $z_t$'s. Report your estimates and the inverse hessian variance-covariance matrix of your estimates.

In [ ]:
# Define z_t using (3)
def z_t(w_t,k_t,alpha):
    return np.log(w_t) - np.log(1-alpha) - alpha*np.log(k_t)

def epsilon_t(z_t, rho,mu):
    z_t_1 = z_t.shift()
    return z_t - rho*z_t_1 - (1-rho)*mu

# Define log likelihood function
def log_lik_norm(w_t,k_t, alpha, rho, mu, sigma):
    epsilonvals = epsilon_t(z_t(w_t,k_t,alpha), rho,mu)[1:]
    ln_pdf_vals = np.log(sts.norm.pdf(epsilonvals,loc=0,scale=sigma))
    return ln_pdf_vals.sum()

# Define Objective Function
def crit(params, w_t, k_t):
    alpha, rho, mu, sigma = params
    log_lik_val = log_lik_norm(w_t,k_t, alpha, rho, mu, sigma)
    return -log_lik_val

### Various Tests
#epsilon_t(z_t(macroseries["w_t"],macroseries["k_t"],alpha_0), rho_0,mu_0)[1:]
#log_lik_norm(macroseries["w_t"],macroseries["k_t"], alpha_0, rho_0, mu_0, sigma_0)
#crit(np.array([alpha_0, rho_0, mu_0, sigma_0]), macroseries["w_t"],macroseries["k_t"])
#macroseries["w_t"],macroseries["k_t"]

In [ ]:
# Run Maximum Likelihood with Gamma
alpha_0, rho_0, mu_0, sigma_0 = .5, .3, .5, .5
params_init = np.array([alpha_0, rho_0, mu_0, sigma_0])
mle_args = (macroseries["w_t"],macroseries["k_t"])
results1 = opt.minimize(crit, params_init, args=mle_args, method = "L-BFGS-B",
                        bounds = ((1e-8, 1-1e-8), (-1+1e-8, 1-1e-8), (1e-8, None), (1e-8, None)))
alpha_MLE, rho_MLE, mu_MLE, sigma_MLE = results1.x

In [ ]:
# Print Results
print('alpha_MLE=', alpha_MLE, ' rho_MLE=', rho_MLE, 'mu_MLE=', mu_MLE, ' sigma_MLE=', sigma_MLE)
print(results1)

# Get Variance Covariance Matrix
vcv_mle1 = results1.hess_inv.todense()
print('VCV(MLE) = ', vcv_mle1)

(b) Use the data $(r_t,k_t)$ and equations (4) and (5) to estimate the four parameters $(\alpha, \rho, \mu, \sigma)$ by maximum likelihood. Given a guess for the parameters $(\alpha, \rho, \mu, \sigma)$, you can use the two variables from the data $(r_t,k_t)$ and (4) to back out a series for $z_t$. You can then use equation (5) to compute the probability of each 
$$ z_t \sim N(\rho z_{t-1} + (1-\rho) \mu, \sigma^2) $$
The maximum likelihood estimate $(\hat{\alpha},\hat{\rho},\hat{\mu},\hat{\sigma})$ maximzies the likelihood function of that normal distribution of $z_t$'s. Report your estimates and the inverse hessian variance-covariance matrix of your estimates.

In [ ]:
# Define z_t using (4)
def z_t2(r_t,k_t,alpha):
    return np.log(r_t) - np.log(alpha) - (alpha-1)*np.log(k_t)

# Same as earlier
def epsilon_t(z_t, rho,mu):
    z_t_1 = z_t.shift()
    return z_t - rho*z_t_1 - (1-rho)*mu

# Define log likelihood function
def log_lik_norm2(r_t,k_t, alpha, rho, mu, sigma):
    epsilonvals = epsilon_t(z_t2(r_t,k_t,alpha), rho,mu)[1:]
    ln_pdf_vals = np.log(sts.norm.pdf(epsilonvals,loc=0,scale=sigma))
    return ln_pdf_vals.sum()

# Define Objective Function
def crit2(params, r_t, k_t):
    alpha, rho, mu, sigma = params
    log_lik_val = log_lik_norm2(r_t,k_t, alpha, rho, mu, sigma)
    return -log_lik_val

### Various Tests
epsilon_t(z_t2(macroseries["r_t"],macroseries["k_t"],alpha_0), rho_0,mu_0)[1:]
log_lik_norm2(macroseries["r_t"],macroseries["k_t"], alpha_0, rho_0, mu_0, sigma_0)
crit(np.array([alpha_0, rho_0, mu_0, sigma_0]), macroseries["r_t"],macroseries["k_t"])
#macroseries["r_t"],macroseries["k_t"]

In [ ]:
# Run Maximum Likelihood with Gamma
alpha_0, rho_0, mu_0, sigma_0 = .5, .3, .5, .5
params_init = np.array([alpha_0, rho_0, mu_0, sigma_0])
mle_args = (macroseries["r_t"],macroseries["k_t"])
results2 = opt.minimize(crit2, params_init, args=mle_args, method = "L-BFGS-B",
                        bounds = ((1e-8, 1-1e-8), (-1+1e-8, 1-1e-8), (1e-8, None), (1e-8, None)))
alpha_MLE2, rho_MLE2, mu_MLE2, sigma_MLE2 = results2.x

In [ ]:
# Print Results
print('alpha_MLE2=', alpha_MLE2, ' rho_MLE2=', rho_MLE2, 'mu_MLE2=', mu_MLE2, ' sigma_MLE2=', sigma_MLE2)
print(results2)

# Get Variance Covariance Matrix
vcv_mle2 = results2.hess_inv.todense()
print('VCV(MLE) = ', vcv_mle2)

(c) According to your estimates from part (a), if investment/savings today is $k_t = 7,500,000$ and the productivity shock today is $z_{t-1} = 10$, what is the probability that the interest rate this period will be greater than $r_t =1$. That is, solve for $Pr(r_t > 1 | \hat{\theta},k_t,c_t, z_{t-1})$. [HINT: Use equation (4) to solve for the $z_t = z^*$ such that $r_t = 1$. Then use (5) to solve for the probability that $z_t > z^*$]